In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

from datetime import datetime, date, timedelta
from time import sleep
import numpy as np
import pandas as pd
import re
from re import search

In [3]:
df = open('Sams', "r")
lines = df.readlines()
df.close()

for index, line in enumerate(lines):
      lines[index] = line.strip()

urls = pd.DataFrame(lines, columns= ["url"])
urls["main_index"] = [i for i in range(len(urls.url))]

In [31]:
# for i, url in enumerate(urls.url):
#     f= open(f"Sams_url_{i}.txt","w+")
#     f.write(url)
#     f.close() 

In [4]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
# driver.maximize_window()

In [5]:
driver.get(urls.loc[urls["main_index"]==2,"url"].values[0])

In [4]:
def review_count(url_index):
    driver.get(urls.loc[urls["main_index"]==url_index,"url"].values[0])
    sleep(6)
    count = driver.find_element_by_xpath("/html/body/jsl/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/span[1]/span/span[1]/span[2]/span[1]/button").text.split(" reviews")[0]
    print(f'python scraper.py --N {count} --i BJ_url_{url_index}.txt')
    


In [6]:
for i in urls.main_index:
    review_count(i)

In [50]:
# review_count()

# Debug

In [23]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from datetime import datetime
import time
import re
import logging
import traceback

GM_WEBPAGE = 'https://www.google.com/maps/'
MAX_WAIT = 10
MAX_RETRY = 5
MAX_SCROLLS = 40

In [15]:
 def __scroll():
        scrollable_div = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.cYB2Ge-oHo7ed') #cYB2Ge-ti6hGc
        # scrollable_div = driver.find_element_by_class_name('section-layout section-scrollbox')
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        #document.getElementsByClassName('section-layout section-scrollbox')[0].scrollBy(0,document.getElementsByClassName('section-layout section-scrollbox')[0].scrollHeight)

__scroll()


In [22]:
def __expand_reviews():
        # use XPath to load complete reviews
        links = driver.find_elements_by_xpath('//button[@class=\'section-expand-review blue-link\']')
        for l in links:
            l.click()
        time.sleep(12)

__expand_reviews()



In [ ]:
def get_reviews(offset):

        # scroll to load reviews

        # wait for other reviews to load (ajax)
        time.sleep(14)

        __scroll()


        # expand review text
        __expand_reviews()

        # parse reviews
        response = BeautifulSoup(driver.page_source, 'html.parser')
        rblock = response.find_all('div', class_='section-review-content')
        parsed_reviews = []
        for index, review in enumerate(rblock):
            if index >= offset:
                parsed_reviews.append(__parse(review))
                print(__parse(review))

        return parsed_reviews

In [167]:
# util function to clean special characters
def __filter_string(str):
        strOut = str.replace('\r', ' ').replace('\n', ' ').replace('\t', ' ')
        return strOut

def __parse(review):
        
        # get address here 
        
        item = {}

        id_review = review.findChild("div")['data-review-id']
        # id_review = review.find('button', class_='section-review-action-menu')['data-review-id'] #OLD
        username = review.find('div', class_='ODSEW-ShBeI-title').find('span').text
        # username = review.find('div', class_='section-review-title').find('span').text


        try:
            # review_text = __filter_string(review.find('span', class_='section-review-text').text)
            review_text = __filter_string(review.find('span', class_='ODSEW-ShBeI-text').text)
        except Exception as e:
            review_text = None

        rating = float(review.find('span', class_='ODSEW-ShBeI-H1e3jb')['aria-label'].split(' ')[1])
        relative_date = review.find('span', class_='ODSEW-ShBeI-RgZmSc-date').text

        # rating = float(review.find('span', class_='section-review-stars')['aria-label'].split(' ')[1])
        # relative_date = review.find('span', class_='section-review-publish-date').text

        try:
            # n_reviews_photos = review.find('div', class_='section-review-subtitle').find_all('span')[1].text
            n_reviews_photos = review.find('div', class_='ODSEW-ShBeI-subtitle').find_all('span')[1].text
            
            metadata = n_reviews_photos.split('\xe3\x83\xbb')
            if len(metadata) == 3:
                n_photos = int(metadata[2].split(' ')[0].replace('.', ''))
            else:
                n_photos = 0

            idx = len(metadata)
            n_reviews = int(metadata[idx - 1].split(' ')[0].replace('.', ''))

        except Exception as e:
            n_reviews = 0
            n_photos = 0

        user_url = review.find('a')['href']

        item['id_review'] = id_review
        item['caption'] = review_text

        # depends on language, which depends on geolocation defined by Google Maps
        # custom mapping to transform into date shuold be implemented
        item['relative_date'] = relative_date

        # store datetime of scraping and apply further processing to calculate
        # correct date as retrieval_date - time(relative_date)
        item['retrieval_date'] = datetime.now()
        item['rating'] = rating
        item['username'] = username
        item['n_review_user'] = n_reviews
        item['n_photo_user'] = n_photos
        item['url_user'] = user_url

        return item


In [168]:
# ODSEW-ShBeI NIyLF-haAclf gm2-body-2
# section-review-content
response = BeautifulSoup(driver.page_source, 'html.parser')
rblock = response.find_all('div', class_='ODSEW-ShBeI NIyLF-haAclf gm2-body-2')
parsed_reviews = []
for index, review in enumerate(rblock):
    if index == 1: #>= offset
        # print(review)
        # print(review.find('span', class_='ODSEW-ShBeI-text').text)
        print(__filter_string(review.find('span', class_='ODSEW-ShBeI-text').text))
        # print(review.findChild("div")['section-review-title'])
        # print(review.find('div', class_='ODSEW-ShBeI-title').find('span').text)
        # parsed_reviews.append(__parse(review))
    #     # print(__parse(review))

# ODSEW-ShBeI-content



I loved my trip I got everything I wanted. I was in & out, no long lines, or waits. My only issue would be that they wouldn't let me use Samsung pay to get my reward points. Can't wait to go back.


In [210]:
# ODSEW-ShBeI NIyLF-haAclf gm2-body-2
# section-review-content
response = BeautifulSoup(driver.page_source, 'html.parser')
rblock = response.find_all('div', class_='ODSEW-ShBeI NIyLF-haAclf gm2-body-2')
parsed_reviews = []
for index, review in enumerate(rblock):
    # if index == 1: #>= offset
        # print(review)
        parsed_reviews.append(__parse(review))
        print(__parse(review))


{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUNxZ1l1c2dnRRAB', 'caption': 'They had a plant sale this afternoon and my wife and I had a good time. We got quite a few things but ... We bought this tomato plant they had priced for 5.41 and they argued that it was 9.75. I wanted to take out a boss but she said it was …', 'relative_date': 'a week ago', 'retrieval_date': datetime.datetime(2021, 6, 16, 21, 9, 14, 551310), 'rating': 2.0, 'username': 'Harold Martinez', 'n_review_user': 0, 'n_photo_user': 0, 'url_user': 'https://www.google.com/maps/contrib/101799531237075069652/reviews?hl=en-US'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUNvbUl1RDB3RRAB', 'caption': "I love Sam's...one of my favorite stores, always a deal to be had! 👌", 'relative_date': 'a month ago', 'retrieval_date': datetime.datetime(2021, 6, 16, 21, 9, 14, 558306), 'rating': 5.0, 'username': 'C B', 'n_review_user': 0, 'n_photo_user': 0, 'url_user': 'https://www.google.com/maps/contrib/109995105564189992382/reviews?hl=en-US'}
{'id_review':

In [183]:
# parsed_reviews[1]

In [176]:
def more_reviews():
        # use XPath to load complete reviews
        #allxGeDnJMl__text gm2-button-alt
        #<button ved="1i:1,t:18519,e:0,p:kPkcYIz-Dtql-QaL1YawDw:1969" jstcache="1202" jsaction="pane.reviewChart.moreReviews" class="gm2-button-alt jqnFjrOWMVU__button-blue" jsan="7.gm2-button-alt,7.jqnFjrOWMVU__button-blue,0.ved,22.jsaction">14 reviews</button>
        #<button aria-label="14 reviews" vet="3648" jsaction="pane.rating.moreReviews" jstcache="1010" class="widget-pane-link" jsan="7.widget-pane-link,0.aria-label,0.vet,0.jsaction">14 reviews</button>
    links = driver.find_elements_by_xpath('')
    print('LINKS HERE', links)
    for l in links:
        l.click()
    time.sleep(12)

more_reviews()

LINKS HERE [<selenium.webdriver.remote.webelement.WebElement (session="62b1bd0238302bf4a993474a12bb5b97", element="c151f367-561d-4d70-8005-0625129d62ac")>]


In [181]:
def __expand_reviews():
        # use XPath to load complete reviews
    links = driver.find_elements_by_xpath('//button[@class=\'section-expand-review blue-link\']')
    for l in links:
        l.click()
    time.sleep(1)


__expand_reviews()
    # //*[@id="pane"]/div/div[1]/div/div/div[2]/div[9]/div[19]/div/div[3]/div[3]/jsl/button

In [208]:
#   https://www.google.com/maps/place/Sam's+Club/@42.7265388,-71.4287465,17z/data=!3m1!4b1!4m5!3m4!1s0x89e3b104e3ab41af:0x774c987611771084!8m2!3d42.7265388!4d-71.4265578
  
def get_address(url):
    driver.get(url)

    time.sleep(10)

    # resp = BeautifulSoup(driver.page_source, 'html.parser')

    # address = __get_address(resp)
    address = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[9]/div[1]/button/div[1]/div[2]/div[1]').text
        
    return address


    # QSFF4-text gm2-body-2


get_address("https://www.google.com/maps/place/Sam's+Club/@42.7265388,-71.4287465,17z/data=!3m1!4b1!4m5!3m4!1s0x89e3b104e3ab41af:0x774c987611771084!8m2!3d42.7265388!4d-71.4265578")

'7 Walmart Blvd, Hudson, NH 03051'

In [206]:
# driver.find_element_by_class_name("QSFF4-text gm2-body-2")
driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[9]/div[1]/button/div[1]/div[2]/div[1]').text

'7 Walmart Blvd, Hudson, NH 03051'

In [ ]:
def __get_address(resp):
        # use XPath to load complete reviews
        
        # button = self.driver.find_elements_by_xpath('/button[@class=\'class="iRxY3GoUYUY__button gm2-hairline-border section-action-chip-button"\']')
        # button.click()
        # time.sleep(2)
        #address = resp.find("div",class_="ugiz4pqJLAG__root ugiz4pqJLAG__one-line-text ugiz4pqJLAG__dense ugiz4pqJLAG__metadata-shown-on-hover-only ugiz4pqJLAG__clickable'").text
        address = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[9]/div[1]/button/div[1]/div[2]/div[1]').text
        return address

In [ ]:
def get_reviews(self, offset):

        # scroll to load reviews

        # wait for other reviews to load (ajax)
        time.sleep(14)

        self.__scroll()


        # expand review text
        self.__expand_reviews()

        # parse reviews
        response = BeautifulSoup(self.driver.page_source, 'html.parser')
        rblock = response.find_all('div', class_='ODSEW-ShBeI NIyLF-haAclf gm2-body-2')
        parsed_reviews = []
        for index, review in enumerate(rblock):
            if index >= offset:
                parsed_reviews.append(self.__parse(review))
                print(self.__parse(review))

        return parsed_reviews